# Multivariate fine-mapping with missing data examples

This notebook applies mvSuSiE on some GTEx genes using two approaches to handle missing data.

## Input

We focus on the 27 genes identified in MASH paper that seems to have different direction of effects in brain vs non-brain tissues. It is of interesting to look at these examples with multivariate fine-mapping.

## Analysis

```
sos run analysis/gtex-v8/20200906_mvSuSiE_GTEx.ipynb \
    --data-dir /project2/compbio/GTEx_eQTL/cis_eqtl_analysis_ready \
    -c midway2.yml -q midway2
```

In [ ]:
[global]
parameter: data_dir = path('~/Documents/GTExV8/Multi_Tissue_Toys')
parameter: wd = path('mvSuSiE_output')
parameter: analysis_units = path('data/27_brain_non_brain_genes_v8.txt')
parameter: prior = path('data/FastQTLSumStats.mash.FL_PC3.rds')
parameter: suffix = 'Multi_Tissues.rds'
regions = [x.strip().split() for x in open(analysis_units).readlines() if x.strip() and not x.strip().startswith('#')]

In [ ]:
[default]
# Only analyze `cis` variants -- cis = N means using N variants around the center column of X matrix  
parameter: cis = 3000
input: [f"{data_dir:a}/{x}.{suffix}" for x in regions if path(f"{data_dir:a}/{x}.{suffix}").exists()], group_by = 1
output: f'{wd:a}/{_input:bn}_mvSuSiE_cis_{cis}.rds'
task: trunk_workers = [4] * 10, trunk_size = 1, walltime = '12h', mem = '8G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr"
    ###Functions to compute MAF and missing genotype rate
    compute_maf <- function(geno){
      f <- mean(geno,na.rm = TRUE)/2
      return(min(f, 1-f))
    }

    compute_missing <- function(geno){
      miss <- sum(is.na(geno))/length(geno)
      return(miss)
    }
    
    mean_impute <- function(geno){
      f <- apply(geno, 2, function(x) mean(x,na.rm = TRUE))
      for (i in 1:length(f)) geno[,i][which(is.na(geno[,i]))] <- f[i]
      return(geno)
    }

    is_zero_variance <- function(x) {
      if (length(unique(x))==1) return(T)
      else return(F)
    }
  
    filter_X <- function(X, missing_rate_thresh, maf_thresh) {
        rm_col <- which(apply(X, 2, compute_missing) > missing_rate_thresh)
        if (length(rm_col)) X <- X[, -rm_col]
        rm_col <- which(apply(X, 2, compute_maf) < maf_thresh)
        if (length(rm_col)) X <- X[, -rm_col]
        rm_col <- which(apply(X, 2, is_zero_variance))
        if (length(rm_col)) X <- X[, -rm_col]
        return(mean_impute(X))
    }

    compute_cov_flash <- function(Y, error_cache = NULL){
        covar <- diag(ncol(Y))
        tryCatch({
        fl <- flashier::flash(Y, var.type = 2, prior.family = c(flashier::prior.normal(), flashier::prior.normal.scale.mix()), backfit = TRUE, verbose.lvl=0)
        if(fl$n.factors==0){
          covar <- diag(fl$residuals.sd^2)
        } else {
          fsd <- sapply(fl$fitted.g[[1]], '[[', "sd")
          covar <- diag(fl$residuals.sd^2) + crossprod(t(fl$flash.fit$EF[[2]]) * fsd)
        }
        if (nrow(covar) == 0) {
          covar <- diag(ncol(Y))
          stop("Computed covariance matrix has zero rows")
        }
        }, error = function(e) {
          if (!is.null(error_cache)) {
            saveRDS(list(data=Y, message=warning(e)), error_cache)
            warning("FLASH failed. Using Identity matrix instead.")
            warning(e)
          } else {
            stop(e)
          }
        })
        s <- apply(Y, 2, sd, na.rm=T)
        if (length(s)>1) s = diag(s)
        else s = matrix(s,1,1)
        covar <- s%*%cov2cor(covar)%*%s
        return(covar)
    }
  
    compute_cov_diag <- function(Y){
        covar <- diag(apply(Y, 2, var, na.rm=T))
        return(covar)
    }

    get_center <- function(k,n) {
      ## For given number k, get the range k surrounding n/2
      ## but have to make sure it does not go over the bounds
      if (is.null(k)) {
          return(1:n)
      }
      start = floor(n/2 - k/2)
      end = floor(n/2 + k/2)
      if (start<1) start = 1
      if (end>n) end = n
      return(start:end)
    }
    dat = readRDS(${_input:r})
    U = readRDS(${prior:r})$Ulist
    weights = rep(1/length(U), length(U))
    prior = mmbr::create_mash_prior(mixture_prior=list(weights=weights, matrices=U))
    resid_Y = compute_cov_flash(dat$y_res)
    X = filter_X(dat$X, 0.1, 0.05)
    X = X[,get_center(${cis}, ncol(X))]
    st = proc.time()
    res = mmbr::msusie(X, dat$y_res, prior_variance=prior, residual_variance=resid_Y, precompute_covariances=T, max_iter = 500)
    res$time = proc.time() - st
    saveRDS(res,${_output:r})